<a href="https://colab.research.google.com/github/Simon-Pu/Temp/blob/master/Hyperparameter_Optimization_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torch import *
%matplotlib inline

from torchvision.models import densenet201
from torchvision.models import resnext50_32x4d
from torchvision.datasets import CIFAR10
from torchvision import transforms

from torch.utils.data import DataLoader
from torch.optim import Adam
import torch.nn.functional as F
from torch.nn import Linear
import torch

print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_properties(0).total_memory)

Tesla P4
7981694976


In [2]:
#model = densenet201(pretrained = True)
model = resnext50_32x4d(pretrained = True)
model.classifier = Linear(1920,10)

# Move model to GPU for faster training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Set model to training mode
model.train()

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/checkpoints/resnext50_32x4d-7cdf4587.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [3]:
# Change the value of bs to change the batch size
bs = 256

train_tfms = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

test_tfms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


train_ds = CIFAR10(root='./data', train=True, download=True, transform=train_tfms)
test_ds = CIFAR10(root='./data', train=False, download=True, transform=test_tfms)

train_dl = DataLoader(train_ds, batch_size = bs)
test_dl = DataLoader(test_ds , batch_size = bs)


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:

loss_func = F.cross_entropy
  # Define function to check model accuracy
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

# Change the value of lr to change the learning rate
lr = 1e-3
optim = Adam(model.parameters(), lr = lr)

# Change the value of epochs to change the number of epochs
epochs = 10

for epoch in range(epochs):
    
    for xb, yb in train_dl:
        
        # .to(device) moves torch.Tensor objects to the GPU for faster training
        
        preds = model(xb.to(device))
        loss = loss_func(preds, yb.to(device))
        acc = accuracy(preds,yb.to(device))

        loss.backward()
        optim.step()
        optim.zero_grad()
        
    print("Loss: " + str(loss.item()) + "\t \t Accuracy: " + str(100 * acc.item()))

Loss: 0.7673590779304504	 	 Accuracy: 72.50000238418579
Loss: 0.5132288932800293	 	 Accuracy: 81.25
Loss: 0.46249261498451233	 	 Accuracy: 83.75000357627869
Loss: 0.3251064419746399	 	 Accuracy: 87.5
Loss: 0.3149528503417969	 	 Accuracy: 86.2500011920929
Loss: 0.3225792348384857	 	 Accuracy: 91.25000238418579
Loss: 0.18286100029945374	 	 Accuracy: 93.75
Loss: 0.2090775966644287	 	 Accuracy: 92.5000011920929
Loss: 0.23269549012184143	 	 Accuracy: 93.75
Loss: 0.19352762401103973	 	 Accuracy: 91.25000238418579


In [6]:
def get_model_accuracy():
  tot_acc = 0
  avg_acc = 0
  
  # Set model to evaluation mode
  model.eval()

  for xbt, ybt in test_dl:

    pred = model(xbt.to(device))
    tot_acc += accuracy(pred,ybt.to(device))

  avg_acc = tot_acc / len(test_dl)
  
  return avg_acc.item()


# Print accuracy of model
print("The average accuracy is: " + str(get_model_accuracy()))

The average accuracy is: 0.85302734375


In [7]:
!pip install bayesian-optimization

In [0]:
def obj_func(lr, bs, epochs):
      
      # We need to round off bs and epochs because Gaussian processes cannot deal with discrete variables 
      bs = int(bs)
      epochs = int(epochs)
      
      train_dl = DataLoader(train_ds, batch_size = bs)
      test_dl = DataLoader(test_ds , batch_size = bs)
      
      optim = Adam(model.parameters(), lr = lr)
      
      for epoch in range(epochs):
    
        for xb, yb in train_dl:
        
            # .to(device) moves torch.Tensor objects to the GPU for faster training
        
            preds = model(xb.to(device))
            loss = loss_func(preds, yb.to(device))
            acc = accuracy(preds,yb.to(device))
        
            loss.backward()
            optim.step()
            optim.zero_grad()
        
        print("Loss: " + str(loss.item()) + "\t \t Accuracy: " + str(100 * acc.item()))

      acc = get_model_accuracy()
      
      return acc

In [9]:
from bayes_opt import BayesianOptimization

# Bounded region of parameter space
pbounds = {'lr': (1e-4, 1e-2), 'bs': (64, 512), 'epochs': (1,25)}

optimizer = BayesianOptimization(
    f=obj_func,
    pbounds=pbounds,
    verbose=2, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

optimizer.maximize(init_points=2, n_iter=3,)

print(optimizer.max)

|   iter    |  target   |    bs     |  epochs   |    lr     |
-------------------------------------------------------------
Loss: 0.12899380922317505	 	 Accuracy: 96.40000462532043
Loss: 0.17769835889339447	 	 Accuracy: 95.20000219345093
Loss: 0.1515907347202301	 	 Accuracy: 95.20000219345093
Loss: 0.10003411024808884	 	 Accuracy: 97.60000705718994
Loss: 0.09661605954170227	 	 Accuracy: 96.8000054359436
Loss: 0.13403838872909546	 	 Accuracy: 96.00000381469727
Loss: 0.05346108227968216	 	 Accuracy: 98.4000027179718
Loss: 0.08634784072637558	 	 Accuracy: 97.60000705718994
Loss: 0.09190244972705841	 	 Accuracy: 96.40000462532043
Loss: 0.038856636732816696	 	 Accuracy: 98.4000027179718
Loss: 0.09273913502693176	 	 Accuracy: 96.00000381469727
Loss: 0.041652459651231766	 	 Accuracy: 98.4000027179718
Loss: 0.12199044227600098	 	 Accuracy: 97.20000624656677
Loss: 0.05599774047732353	 	 Accuracy: 97.60000705718994
Loss: 0.0832596868276596	 	 Accuracy: 96.8000054359436
Loss: 0.04410770162940025	

In [0]:
from bayes_opt import BayesianOptimization

# Bounded region of parameter space
pbounds = {'lr': (1e-4, 1e-2), 'bs': (64, 512), 'epochs': (1,25)}

optimizer = BayesianOptimization(
    f=obj_func,
    pbounds=pbounds,
    verbose=2, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

optimizer.maximize(init_points=2, n_iter=3,)

print(optimizer.max)